# Movie Recommendation System: context features, retrieval model

## Install libraries

In [1]:
!pip install -q tensorflow-recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 1.4 MB/s eta 0:00:00


## Import libraries

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import pandas as pd
import numpy as np


import tensorflow as tf
import tensorflow_recommenders as tfrs

## Constants

In [3]:
data_Path = '/content/drive/MyDrive/Colab Notebooks/Recommended_Systems_ Movie/ml-100k'


## Reading Data

In [4]:
user_info_path = os.path.join(data_Path, 'u.info')

user_info = pd.read_csv(user_info_path, header=None)
user_info.head()

,0
0,943 users
1,1682 items
2,100000 ratings


### Users Data:

In [5]:
user_data_path = os.path.join(data_Path, 'u.data')

#Reading users file:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
users_df = pd.read_csv(user_data_path, sep='\t', names=column_names)
# Checking shape of users files and head
print(users_df.shape)
users_df.head()

(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


### Movies Data:

In [6]:
columns = "item_id | title | release date | video release date | "\
             "IMDb URL | unknown | Action | Adventure | Animation | Children's | "\
              "Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | "\
               "Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western | "

column_names_2 = columns.split(' | ')
movie_data_path = os.path.join(data_Path, 'u.item')

# Reading the movie data
movie_df = pd.read_csv(movie_data_path, sep='|', header=None, names=column_names_2, encoding='latin-1')
movie_df.drop(movie_df.columns[-1], axis=1, inplace=True)
# Checking shape of movie data and look first 5 rows
print(movie_df.shape)
movie_df.head()

(1682, 24)


,item_id,title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [7]:
movie_df = movie_df[['item_id', 'title']]
movie_df.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


### Merger Movie and User data

In [8]:
# Combining the data on same column
df= pd.merge(users_df, movie_df, on= 'item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [9]:
df.shape

(100000, 5)

In [10]:
df['user_id'] = df['user_id'].values.astype(str)
df = tf.data.Dataset.from_tensor_slices(dict(df))

In [11]:
ratings = df.map(lambda x: {
    "movie_title": x["title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
})
movies = df.map(lambda x: x["title"])

prepare feature vocabularies

In [12]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

## Model definition

### Query model

In [13]:
class UserModel(tf.keras.Model):

  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
      ])
      self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
      )

      self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    ], axis=1)

## Candidate model

In [14]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(movies)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)

## Combined model

In [15]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

### Prepare the data

In [16]:
AUTOTUNE = tf.data.AUTOTUNE

tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048).cache().prefetch(buffer_size=AUTOTUNE)
cached_test = test.batch(4096).cache().cache().prefetch(buffer_size=AUTOTUNE)

### Baseline: no timestamp features

In [ ]:
model = MovielensModel(use_timestamps=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
40/40 [==============================] - 435s 11s/step - factorized_top_k/top_1_categorical_accuracy: 0.0150 - factorized_top_k/top_5_categorical_accuracy: 0.0154 - factorized_top_k/top_10_categorical_accuracy: 0.0155 - factorized_top_k/top_50_categorical_accuracy: 0.0170 - factorized_top_k/top_100_categorical_accuracy: 0.0190 - loss: 14574.0117 - regularization_loss: 0.0000e+00 - total_loss: 14574.0117
Epoch 2/3
40/40 [==============================] - 397s 10s/step - factorized_top_k/top_1_categorical_accuracy: 0.0026 - factorized_top_k/top_5_categorical_accuracy: 0.0026 - factorized_top_k/top_10_categorical_accuracy: 0.0030 - factorized_top_k/top_50_categorical_accuracy: 0.0057 - factorized_top_k/top_100_categorical_accuracy: 0.0087 - loss: 14151.3611 - regularization_loss: 0.0000e+00 - total_loss: 14151.3611
Epoch 3/3
5/5 [==============================] - 92s 18s/step - factorized_top_k/top_1_categorical_accuracy: 8.5000e-04 - factorized_top_k/top_5_categorical_accuracy:

### Capturing time dynamics with time features

In [18]:
model = MovielensModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
40/40 [==============================] - 440s 11s/step - factorized_top_k/top_1_categorical_accuracy: 0.0163 - factorized_top_k/top_5_categorical_accuracy: 0.0163 - factorized_top_k/top_10_categorical_accuracy: 0.0164 - factorized_top_k/top_50_categorical_accuracy: 0.0186 - factorized_top_k/top_100_categorical_accuracy: 0.0206 - loss: 14548.0425 - regularization_loss: 0.0000e+00 - total_loss: 14548.0425
Epoch 2/3
40/40 [==============================] - 445s 11s/step - factorized_top_k/top_1_categorical_accuracy: 0.0038 - factorized_top_k/top_5_categorical_accuracy: 0.0039 - factorized_top_k/top_10_categorical_accuracy: 0.0043 - factorized_top_k/top_50_categorical_accuracy: 0.0068 - factorized_top_k/top_100_categorical_accuracy: 0.0099 - loss: 13952.6100 - regularization_loss: 0.0000e+00 - total_loss: 13952.6100
Epoch 3/3
5/5 [==============================] - 105s 20s/step - factorized_top_k/top_1_categorical_accuracy: 0.0010 - factorized_top_k/top_5_categorical_accuracy: 0.